# DVE Rental SQL Problems

## Setup

In [3]:
import psycopg2
import pandas as pd
import numpy as np
import pymysql
import sys

In [6]:
try:
    conn = psycopg2.connect("dbname='dvdrental' user='' host='localhost' password=''")
except:
    print("I am unable to connect to the database")

In [7]:
category = pd.read_sql_query('SELECT * from category',con=conn)
film_category = pd.read_sql_query('SELECT * from film_category',con=conn)
film = pd.read_sql_query('SELECT * from film',con=conn)
language = pd.read_sql_query('SELECT * from language',con=conn)
film_actor = pd.read_sql_query('SELECT * from film_actor',con=conn)
inventory = pd.read_sql_query('SELECT * from inventory',con=conn)
rental = pd.read_sql_query('SELECT * from rental',con=conn)
payment = pd.read_sql_query('SELECT * from payment',con=conn)
staff = pd.read_sql_query('SELECT * from staff',con=conn)
actor = pd.read_sql_query('SELECT * from actor',con=conn)
customer = pd.read_sql_query('SELECT * from customer',con=conn)
address = pd.read_sql_query('SELECT * from address',con=conn)
city = pd.read_sql_query('SELECT * from city',con=conn)
country = pd.read_sql_query('SELECT * from country',con=conn)
store = pd.read_sql_query('SELECT * from store',con=conn)

![](_pic/DVD-Rental-ER-Diagram.png)

# Question, Idea & Solution

---

> ### Get a list of actors with the first name Chris, Cameron, or Cuba.


**> idea:**
Use the ```SELECT``` & ```WHERE``` command to filter out the ideal rows.
 
**> solution:**

```
    SELECT * from actor where first_name in ('Chris', 'Cameron', 'Cuba'); 
```    

**> note:**

* `IN` lets you specify a lot of values that you would otherwise join together with an `OR` statement
* We can test for `NULL` with `IS NULL`. If we want to filter out '<NA>' rows, we can use `IS NOT NULL` in `WHERE`.
* `WHERE` operators include:

| Operator | Description |
|:---:|:---|
| = | Equal |
| > | Greater than |
| < | Less than |
| >= | Greater than or equal |
| <= | Less than or equal |
| <> or != | Not equal |
| AND | Logical operator AND |
| OR | Logical operator OR |

In [5]:
actor[actor['first_name'].isin(['Chris','Cameron', 'Cuba'])]

,actor_id,first_name,last_name,last_update
14,15,Cuba,Olivier,2013-05-26 14:47:57.620
23,24,Cameron,Streep,2013-05-26 14:47:57.620
62,63,Cameron,Wray,2013-05-26 14:47:57.620
98,98,Chris,Bridges,2013-05-26 14:47:57.620
111,111,Cameron,Zellweger,2013-05-26 14:47:57.620
118,118,Cuba,Allen,2013-05-26 14:47:57.620
159,160,Chris,Depp,2013-05-26 14:47:57.620
188,189,Cuba,Birch,2013-05-26 14:47:57.620


In [6]:
actor.query('first_name=="Chris" | first_name=="Cameron" | first_name=="Cuba" ')

,actor_id,first_name,last_name,last_update
14,15,Cuba,Olivier,2013-05-26 14:47:57.620
23,24,Cameron,Streep,2013-05-26 14:47:57.620
62,63,Cameron,Wray,2013-05-26 14:47:57.620
98,98,Chris,Bridges,2013-05-26 14:47:57.620
111,111,Cameron,Zellweger,2013-05-26 14:47:57.620
118,118,Cuba,Allen,2013-05-26 14:47:57.620
159,160,Chris,Depp,2013-05-26 14:47:57.620
188,189,Cuba,Birch,2013-05-26 14:47:57.620


In [ ]:
actor[(actor['first_name'] == "Chris") | (actor['first_name'] == "Cameron")]

**pandas note**

* we can filter the dataframe using `.query('colname2>10 & colname2=="abc"')`. For or, using `|` 

---

> ### What are the different rental durations that the store allows?

**> idea:**
Use `DISTINCT` operator together in `SELECT`

**> solution:**
```
select distinct rental_duration from film;
```




In [7]:
film.rental_duration.unique()

array([7, 5, 6, 4, 3])

---
> ### How many films are rated NC-17? How many are rated PG or PG-13?

**> idea:**
filter rows from films that are rated "NC-17", and use count command to see how many rows there are.
If we want to check "How many are rated PG or PG-13?", in the where command, we can use `in` to filter rows that are in `('PG', 'PG-13')`

**> solution:**
```
select count(*) from film where rating in ('PG','PG-13');
```

In [8]:
film[film['rating'].isin(['PG','PG-13'])].groupby(['rating']).agg({'film_id':'count'}).reset_index().rename(columns={'film_id': 'count'})

,rating,count
0,PG,194
1,PG-13,223


**pandas note**

* To summarise after groupby, we can use `.agg({'colname':'count'})`
* To rename colnames, we can use `.rename(columns={'oldName1': 'newName1', 'oldName2': 'newName2'})`. Remember to use `columns` with s.

Here are a list of method that we can use for `agg`

| method |  |
|:---:|:---|
| 'sum' or np.sum | return the sum |
| np.mean | Return the mean |
| 'count' | return the count |
| 'nunique' | Return the count of a number of different values |
| 'max' | Return the maximum value |
| 'min' | Return the minimum value |
| np.std | Return the standard deviation |
| np.var | Return the variance |


---

> ### How many different customers have entries in the rental table?

**> idea:**
Use distinct to get the unique id from the field `customer_id`, and then combine with `count` command to get the count number.


**> solution:**
```
select count(distinct customer_id) from rental;
```

In [57]:
rental['customer_id'].nunique()

599

---

> ### What are the IDs of the last 5 customers to return a rental?

**> idea:**
order the `rental` table by `return_date` in descending order and get the `customer_id` of the first 3 rows


**> solution:**

```
select customer_id, return_date from rental where customer_id=251 order by return_date DESC;
```

```
select customer_id, count(*), max(return_date) from rental group by customer_id;
```


```
select customer_id, return_date from rental where return_date is not null order by return_date DESC limit 5;
```

**> better solution:**
Since a customer_id can show up twice in the first five rows. To deduplicated this, we can do the following. Even though the result is the same, but it's a better solution.
```
select customer_id, recent_date from (
  select customer_id, max(return_date) as recent_date from rental where return_date is not null group by customer_id 
) as tbl1 order by recent_date DESC limit 5;
```



**> note:**

* We can add `DESC` after `order by` to get the rows in descending order. If not specify, it will be ascending order.
* If we want to filter out '<NA>' rows, we can use `IS NOT NULL` in `WHERE`.

In [61]:
rental.groupby(['customer_id']).agg({'return_date':'max'}).reset_index().sort_values(by=['return_date'], ascending=False).head(5)

,customer_id,return_date
465,466,2005-09-02 02:35:22
194,195,2005-09-02 02:19:33
186,187,2005-09-02 01:28:33
74,75,2005-09-01 23:43:24
262,263,2005-09-01 22:27:31


**pandas note**

* to sort values, we can use `.sort_values(by=['colname1','colnames2'], ascending=False)` to return value by descending order. 

---

> ### Does the average replacement cost of a film differ by rating?

**> idea:**
We will want to `group by` rating and calculate the average replacement cost.

**> solution:**
```
select rating, avg(replacement_cost) from film group by rating;
```


**> note:**

* All columns except the columns applied with some calculation in the `SELECT` part of the statement have to be in the GROUP BY part, or you'll get an error.
* The [aggregate function](https://www.postgresql.org/docs/9.5/static/functions-aggregate.html) that can be used with `GROUP BY` include:

| Name | Description |
|:---:|:---|
| avg() | Return the average value of the argument |
| count() | Return a count of the number of rows returned |
| count(distinct) | Return the count of a number of different values |
| max() | Return the maximum value |
| min() | Return the minimum value |
| sum() | Return the sum |
| stddev_pop | Return the population standard deviation |
| stddev(), stddev_samp() | Return the sample standard deviation |
| var_pop() | Return the population standard variance |
| variance(), var_samp() | Return the sample variance |
| array_agg() | input arrays concatenated into array of one higher dimension  |
| json_agg | aggregates values as a JSON array | 

In [62]:
film.groupby(['rating']).agg({'replacement_cost':np.mean}).reset_index()

,rating,replacement_cost
0,G,20.124831
1,NC-17,20.137619
2,PG,18.959072
3,PG-13,20.402556
4,R,20.231026


---

> ### Select film title that have "Dragon" in them.

**> idea:**
Using some command with `where` to filter our title with Dragon in them. It turns out the command we need is `similar to` combining with some wildcards.


**> solution:**
```
select * from film where title similar to '[C|F]%Dragon%';
```


**> note:**

* `%` : The percent sign represents zero, one, or multiple characters
* `_` : The underscore represents a single character
* `'[bsp]%'`: The following SQL statement selects the target value starting with "b", "s", or "p"
* `'[!bsp]%'`: The following SQL statement selects the target value NOT starting with "b", "s", or "p"
    * We can also use `NOT LIKE '[bsp]%'`
* `'[a-c]%'`: The following SQL statement selects the target value starting with "a", "b", or "c"
* `'[a|c]%'`: denotes alternation (either of two alternatives). match value starting with either "a" or "c".
* For more matching command, check [[here]](https://www.postgresql.org/docs/9.0/static/functions-matching.html).

In [63]:
film[film['title'].str.contains("Dragon")]

,film_id,title,description,release_year,language_id,rental_duration,rental_rate,length,replacement_cost,rating,last_update,special_features,fulltext
128,124,Casper Dragonfly,A Intrepid Documentary of a Boat And a Crocodi...,2006,1,3,4.99,163,16.99,PG-13,2013-05-26 14:50:58.951,[Trailers],'boat':8 'casper':1 'chase':14 'crocodil':11 '...
251,250,Dragon Squad,A Taut Reflection of a Boy And a Waitress who ...,2006,1,4,0.99,170,26.99,NC-17,2013-05-26 14:50:58.951,"[Deleted Scenes, Behind the Scenes]",'ancient':18 'boy':8 'china':19 'dragon':1 'mu...
252,251,Dragonfly Strangers,A Boring Documentary of a Pioneer And a Man wh...,2006,1,6,4.99,133,19.99,NC-17,2013-05-26 14:50:58.951,"[Trailers, Behind the Scenes]",'bore':4 'documentari':5 'dragonfli':1 'man':1...
299,299,Factory Dragon,A Action-Packed Saga of a Teacher And a Frisbe...,2006,1,4,0.99,144,9.99,PG-13,2013-05-26 14:50:58.951,"[Trailers, Commentaries, Deleted Scenes]",'action':5 'action-pack':4 'desert':22 'dragon...


**pandas note**

* we can use `.str.contains("pattern")` within filter to filter by string pattern.

---

> ### Which store (store_id) has the most customers whose first name starts with M?

**> idea:**
We will probably need to filter out customer with 'first_name' starting with M, then group by `store_id`, and then count the distinct `customer_id`. (Actually, since `customer_id` is the primary key of the table, we don't need `distinct`.)


**> solution:**
```
select store_id, count(distinct customer_id) from customer where first_name similar to 'M%' group by store_id order by count(*) DESC;
```

In [80]:
customer[customer['first_name'].str.startswith("M")].groupby('store_id').agg({'customer_id':'count'}).reset_index().rename(columns={'customer_id':'count'})

,store_id,count
0,1,30
1,2,21


---

> ### Are there any customers with the same last name?

**> idea:**
We will probably want to `group by` `last_name` to check what are the `last_name` that is used ore than once. To do this, we will need to combine with `Having`

**> solution:**
```
select last_name, count(*) from customer group by last_name having count(*)>1;
```
It turns out that there is no customers with the same last name.

In [82]:
customer.groupby(['last_name'])['customer_id'].count().reset_index().rename(columns={'customer_id':'count'}).query("count>1")

,last_name,count


**pandas note**

* we can filter the dataframe using `.query('colname>10')`

---

> ### Which film (id) has the most actors?

**> idea:**
To do this, we will need to use `film_actor` table, `group by` `film_id` and count `actor_id`. Also, we'll want to sort by the count value.

**> solution:**
```
select film_id, count(actor_id) from film_actor group by film_id order by count(actor_id) DESC;
```

In [92]:
film_actor.groupby(['film_id'])['actor_id'].count().reset_index().rename(columns={'actor_id':'count'}).sort_values(by=['count'], ascending = False).head()

,film_id,count
505,508,15
187,188,13
711,714,13
86,87,13
145,146,13


---

> ### What films are actors with ids 129 and 195 in together?

**> idea:**
What we will need to do is first select films that `actor_id 129`is in. Then, out of those films, we then filter out films that `actor_id 195` is also in. Because we need to do twice, we'll need to use subquery

**> solution:**

```
select film_id from film_actor where film_id IN (select film_id from film_actor where actor_id=129) and actor_id=195;
```

In [97]:
a=film_actor.query('actor_id==129')
b=film_actor.query('actor_id==195')
c=pd.merge(a,b,on='film_id',how='left')

c[(c.actor_id_y).notnull()]

,actor_id_x,film_id,last_update_x,actor_id_y,last_update_y
15,129,520,2006-02-15 10:05:03,195.0,2006-02-15 10:05:03
29,129,962,2006-02-15 10:05:03,195.0,2006-02-15 10:05:03


**pandas note**

* to drop rows of Pandas DataFrame whose value in certain columns is NaN. We can use `df[df.colname.notnull()]` or `df[~df.colname.isnull()]`
* to merge dataframes, we can use `pd.merge(df1,df2,  on='ad_id', how='left')`. 
    * how : {'left', 'right', 'outer', 'inner'}, default 'inner'

---

> ### How many actors are in more films than actor id 47? 

**> idea:**
We'll firstly need to know how many film do actor 47 perform in. We then need to know how many films do each actors perform in. Then, we filter out the actors that performs more films than actor 47.

Having the above table, we then count how many rows in the table.

**> solution:**
```
select count(*) from 
    (Select actor_id, count(*) from film_actor group by actor_id 
      having 
          count(*) > (select count(*) from film_actor where actor_id=47)
    ) tbl1;
```

**> note:**

* If we want to select from some temporary generated result set, we'll need to give our subquery a name.

In [103]:
id47 = film_actor.query('actor_id==47')['film_id'].count()

film_actor.groupby('actor_id').agg({'film_id':'nunique'}).reset_index().rename(columns={'film_id':'count'}).query('count>@id47').head()


,actor_id,count
1,2,25
4,5,29
6,7,30
8,9,25
10,11,25


**pandas note**

* to pass a variable into a query, we can use `.query('colname>@variable')`

---

> ### Join the customer and payment tables together with an inner join; select customer id, name, amount, and date and order by customer id. Then join the staff table to them as well to add the staff's name.

**> idea:**
This is just to practice some command for inner join

**> solution:**
```
select * from 
  staff as s, 
  (select c.customer_id, c.first_name, c.last_name, p.amount, c.create_date, p.staff_id 
  from customer as c, payment as p where c.customer_id=p.customer_id 
  order by customer_id) tbl1 
where s.staff_id=tbl1.staff_id; 
```
```
SELECT
 customer.customer_id,
 customer.first_name customer_first_name,
 customer.last_name customer_last_name,
 staff.first_name staff_first_name,
 staff.last_name staff_last_name,
 amount,
 payment_date
FROM
 customer
INNER JOIN payment ON payment.customer_id = customer.customer_id
INNER JOIN staff ON payment.staff_id = staff.staff_id
ORDER BY
 customer.customer_id;
```

**> note:**

* We can use `WHERE r1.id=r2.id` or `INNER JOIN r2 on r1.id=r2.id` to do a inner join
* When using `INNER JOIN` and `WHERE` together to filter something, `WHERE` should be used after the `INNER JOIN` command.

In [12]:
pd.merge(customer,payment, on='customer_id',how='inner').rename(columns={'first_name':'customer_first','last_name':'customer_last'}).merge(staff,on='staff_id')[['customer_id','customer_first','customer_last','amount','create_date','staff_id','first_name','last_name']].head(5)


,customer_id,customer_first,customer_last,amount,create_date,staff_id,first_name,last_name
0,524,Jared,Ely,1.99,2006-02-14,1,Mike,Hillyer
1,524,Jared,Ely,2.99,2006-02-14,1,Mike,Hillyer
2,524,Jared,Ely,2.99,2006-02-14,1,Mike,Hillyer
3,524,Jared,Ely,5.99,2006-02-14,1,Mike,Hillyer
4,524,Jared,Ely,6.99,2006-02-14,1,Mike,Hillyer


---

> ### Create a list of addresses that includes the name of the city instead of an ID number and the name of the country as well.

**> idea:**
We'll want to do a inner join using address and city, also address and country

**> solution:**
```
select 
  address, 
  address2,
  city,
  country
from 
  address as a
inner join city on a.city_id=city.city_id
inner join country on city.country_id=country.country_id;
```

In [15]:
address.merge(city,on='city_id',how='left').merge(country,on='country_id',how='left')[['address','city','country']].head(5)


,address,city,country
0,47 MySakila Drive,Lethbridge,Canada
1,28 MySQL Boulevard,Woodridge,Australia
2,23 Workhaven Lane,Lethbridge,Canada
3,1411 Lillydale Drive,Woodridge,Australia
4,1913 Hanoi Way,Sasebo,Japan


---

> ### Which two actors have been in the most films together?

**> idea:**
It is a harder problem. To start, we will probably need to get a dataframe with actor_id 1 and actor_id 2 and the film_id. With this dataframe, we can then count how many rows do each of the two actor_id appears together in the dataframe. We can get the count by using `group by`.

**> solution:**
```
SELECT a.actor_id, b.actor_id, count(*)
FROM film_actor a, film_actor b -- join the table to itself
WHERE a.film_id=b.film_id -- on the film id
      AND a.actor_id > b.actor_id -- avoid duplicates and matching to the same actor
GROUP BY a.actor_id, b.actor_id
ORDER BY count(*) DESC;
```

In [16]:
film_actor.head()

,actor_id,film_id,last_update
0,1,1,2006-02-15 10:05:03
1,1,23,2006-02-15 10:05:03
2,1,25,2006-02-15 10:05:03
3,1,106,2006-02-15 10:05:03
4,1,140,2006-02-15 10:05:03


In [19]:
film_actor.merge(film_actor,on='film_id').query('actor_id_x<actor_id_y').groupby(['actor_id_x','actor_id_y']).agg({'film_id':'count'}).reset_index().sort_values(by='film_id',ascending=False).head(5)


,actor_id_x,actor_id_y,film_id
2543,27,60,7
1478,15,198,6
8504,114,168,6
3471,37,188,6
6794,83,161,6


---

> ### Get a list of the names of customers who have spent more than $150, along with their total spending.

**> idea:**
We can firstly get the `customer_id` that has spent more than $150 using payment. Then we join the result with the `customer` t o get the name.

We can also firstly join `customer` to get all the names and then `group by` to filter out all the customers with payment over $150. The first version is more efficient, since it filter out before doing the join.

**> solution:**

```
select c.first_name, c.last_name, c.customer_id, tbl1.sum
from customer as c,
  (select 
    p.customer_id, sum(amount) 
  from payment as p 
  group by customer_id having sum(amount) > 150) tbl1
where c.customer_id = tbl1.customer_id
order by sum DESC;
```

```
select 
  c.first_name,
  c.last_name,
  sum(amount)
from 
  customer as c,
  payment as p
where 
  c.customer_id=p.customer_id 
group by  
  c.first_name, c.last_name
having 
  sum(amount) > 150
order by 
  sum(amount) DESC;
```

In [17]:
payment.groupby(['customer_id']).agg({'amount':'sum'}).reset_index().query('amount>150').merge(customer, on='customer_id', how='inner')[['customer_id','first_name','last_name','amount']].sort_values(by='amount',ascending=False)


,customer_id,first_name,last_name,amount
2,148,Eleanor,Hunt,211.55
19,526,Karl,Seal,208.58
4,178,Marion,Snyder,194.61
0,137,Rhonda,Kennedy,191.62
1,144,Clara,Shaw,189.60
13,459,Tommy,Collazo,183.63
5,181,Ana,Bradley,167.67
12,410,Curtis,Irby,167.62
7,236,Marcia,Dean,166.61
11,403,Mike,Way,162.67


In [25]:
connection = pymysql.connect(host='localhost',
                             user='root',
                             password='',
                             db='world')

In [26]:
city=pd.read_sql_query('SELECT * from city',con=connection)
country=pd.read_sql_query('SELECT * from country',con=connection)
language=pd.read_sql_query('SELECT * from country',con=connection)

In [11]:
city.head()

,ID,Name,CountryCode,District,Population
0,1,Kabul,AFG,Kabol,1780000
1,2,Qandahar,AFG,Qandahar,237500
2,3,Herat,AFG,Herat,186800
3,4,Mazar-e-Sharif,AFG,Balkh,127800
4,5,Amsterdam,NLD,Noord-Holland,731200


In [12]:
country.head()

,Code,Name,Continent,Region,SurfaceArea,IndepYear,Population,LifeExpectancy,GNP,GNPOld,LocalName,GovernmentForm,HeadOfState,Capital,Code2
0,ABW,Aruba,North America,Caribbean,193.0,NaN,103000,78.4,828.0,793.0,Aruba,Nonmetropolitan Territory of The Netherlands,Beatrix,129.0,AW
1,AFG,Afghanistan,Asia,Southern and Central Asia,652090.0,1919.0,22720000,45.9,5976.0,NaN,Afganistan/Afqanestan,Islamic Emirate,Mohammad Omar,1.0,AF
2,AGO,Angola,Africa,Central Africa,1246700.0,1975.0,12878000,38.3,6648.0,7984.0,Angola,Republic,José Eduardo dos Santos,56.0,AO
3,AIA,Anguilla,North America,Caribbean,96.0,NaN,8000,76.1,63.2,NaN,Anguilla,Dependent Territory of the UK,Elisabeth II,62.0,AI
4,ALB,Albania,Europe,Southern Europe,28748.0,1912.0,3401200,71.6,3205.0,2500.0,Shqipëria,Republic,Rexhep Mejdani,34.0,AL


In [13]:
language.head()

,Code,Name,Continent,Region,SurfaceArea,IndepYear,Population,LifeExpectancy,GNP,GNPOld,LocalName,GovernmentForm,HeadOfState,Capital,Code2
0,ABW,Aruba,North America,Caribbean,193.0,NaN,103000,78.4,828.0,793.0,Aruba,Nonmetropolitan Territory of The Netherlands,Beatrix,129.0,AW
1,AFG,Afghanistan,Asia,Southern and Central Asia,652090.0,1919.0,22720000,45.9,5976.0,NaN,Afganistan/Afqanestan,Islamic Emirate,Mohammad Omar,1.0,AF
2,AGO,Angola,Africa,Central Africa,1246700.0,1975.0,12878000,38.3,6648.0,7984.0,Angola,Republic,José Eduardo dos Santos,56.0,AO
3,AIA,Anguilla,North America,Caribbean,96.0,NaN,8000,76.1,63.2,NaN,Anguilla,Dependent Territory of the UK,Elisabeth II,62.0,AI
4,ALB,Albania,Europe,Southern Europe,28748.0,1912.0,3401200,71.6,3205.0,2500.0,Shqipëria,Republic,Rexhep Mejdani,34.0,AL


> ### Find the largest country in terms of population in each continent

**> idea:**
1. First have a table with column `continent, country, population`
2. Get the country with the largest population for each continent from the table.

The second part is tricky. We can not just `group by` continent and then select the max country by population. It's not that straight forward. 

We probably need to join the table by itself and then use some condition to get the conutry with the largest population. It's actually the same question as the problem1 from leetcode above.

**> solution:**
```
select * from city;
```

```{sql, connection=con_mysql}
select 
    cn.Continent,
    cn.Name,
    sum(ci.Population) as population
from 
  city as ci
inner join country as cn on ci.CountryCode=cn.Code
group by
  ci.CountryCode
order by
  cn.Continent,
  population DESC;
```


```
select 
  tbl1.Continent,
  tbl1.Name,
  tbl1.population
from 
  (select 
      cn.Continent,
      cn.Name,
      sum(ci.Population) as population
  from 
    city as ci
  inner join country as cn on ci.CountryCode=cn.Code
  group by
    ci.CountryCode
  order by
    cn.Continent,
    population DESC) as tbl1
left join
   (select 
        cn.Continent,
        cn.Name,
        sum(ci.Population) as population
    from 
      city as ci
    inner join country as cn on ci.CountryCode=cn.Code
    group by
      ci.CountryCode
    order by
      cn.Continent,
      population DESC) as tbl2 
  on 
    tbl1.Continent=tbl2.Continent and
    tbl2.population > tbl1.population
where 
  tbl2.population is NULL;
```

**> note:**

* `inner join` should be before `group by`

> ### Find the largest country (by area) in each continent, show the continent, the name and the area:

**> idea:**
1. We will need a table with column `continent, name, area`.
2. We then want to get the largest country by area for each continent. 


**> solution:**
```
select * from country;
```

```
select 
  cn1.Continent, cn1.Name, cn1.SurfaceArea
from
  country as cn1
left join
  country as cn2 
on 
  cn1.Continent = cn2.Continent and -- for every continent in cn1, it will join with the same continent in cn2.
  cn2.SurfaceArea > cn1.SurfaceArea -- but we don't want to join country in each continent. We only want to join with the country which SurfaceArea is larger than the country in the left table. 
where 
  cn2.SurfaceArea is NULL; -- If the country in cn1 is the largest, then it will have no country from the right table to join. Therefore, it is the country we want. 
```

```
-- https://www.xaprb.com/blog/2006/12/07/how-to-select-the-firstleastmax-row-per-group-in-sql/
-- correlated subquery
SELECT -- not sure why -- ???
  Continent, Name, SurfaceArea
FROM country x
WHERE SurfaceArea >= ALL
    (SELECT SurfaceArea FROM country y
    WHERE y.continent=x.continent
    AND SurfaceArea>0);
```

In [28]:
country.sort_values(by='SurfaceArea',ascending=False).groupby('Continent').head(1)[['Continent','Name','SurfaceArea']]

,Continent,Name,SurfaceArea
181,Europe,Russian Federation,17075400.0
11,Antarctica,Antarctica,13120000.0
37,North America,Canada,9970610.0
41,Asia,China,9572900.0
30,South America,Brazil,8547403.0
14,Oceania,Australia,7741220.0
184,Africa,Sudan,2505813.0


> ### List each continent and the name of the country that comes first alphabetically.

**> idea:**
1. we need to have a table have columns `continent, NAME`
2. keep only the country with its name comes first alphabetically

**> solution:**
```
select 
  cn1.Continent,
  cn1.Name 
from 
  country as cn1
left join
  country as cn2
on 
  cn1.Continent=cn2.Continent and
  cn2.Name < cn1.Name
where 
  cn2.Name is NULL;
```
```
SELECT Continent, Name
FROM country x
WHERE Name <= ALL(SELECT Name FROM country y WHERE y.Continent = x.Continent)
```

> ### Find the continents where all countries have a population <= 25000000. Then find the names of the countries associated with these continents. Show name, continent and population.

**> idea:**
1. we need a table with columns `Continent, country, population`

**> solution:**
```
-- ???
SELECT Name, Continent, population
FROM country x
WHERE 25000000  > ALL(SELECT population FROM country y WHERE x.Continent = y.Continent AND y.population > 0)

```

> ### Which countries have a GDP greater than every country in Europe? [Give the name only.] (Some countries may have NULL gdp values)

**> idea:**
1. select country name and then filter the rows using ALL command, continent!='Europe' and gdp!=NULL


**> solution:**
```
select Name
from country
where 
  GNP > ALL(select GNP from country where Continent="Europe" and GNP is NOT NULL) and 
  Continent!='Europe' and 
  GNP is NOT NULL;
```

---

> **Read csv**

In [ ]:
# no index column
df = pd.read_csv('data/final_set.csv')

# use the first column as index
df = pd.read_csv('data/final_set.csv', index_col=0)

> **Read Excel**

In [ ]:
df = pd.read_excel('data/WHR2018Chapter2OnlineData.xls', sheet_name='Table2.1')

> **Drop Columns**

In [ ]:
df.drop(['column1', 'column2'], axis=1, inplace=True)

In [ ]:
df = df.drop(['column1', 'column2'], axis=1)

> **Selecting columns in a pandas dataframe**

In [ ]:
df1 = df[['a','b']]

In [ ]:
df1 = df.iloc[:,0:2] # Remember that Python does not slice inclusive of the ending index.

In [ ]:
df1 = df.iloc[0,0:2].copy() # To avoid the case where changing df1 also changes df

[stackoverflow](https://stackoverflow.com/questions/11285613/selecting-columns-in-a-pandas-dataframe)

> **Check NA percentage for each column**

In [ ]:
df.isnull().sum()/df.shape[0]

> **Filter NA rows in a specific column**

In [ ]:
df = df[df.colname.notnull()]

> **Filter out rows with any NA**

In [ ]:
df[df.isnull().any(axis=1)]

> **Filter with multiple conditions**

In [ ]:
df[(df['country'].isin(final_set['country'])) & (df['year']==2008)].copy()
df[(df['country'].isin(final_set['country'])) | (df['year']==2008)].copy()

> **fill na in pandas**

In [ ]:
df.fillna('someNewValue', inplace=True)

In [ ]:
df = df.fillna('someNewValue')

> **Save as csv**

In [ ]:
df.to_csv('name.csv', sep=',', index = False)

> **Converting a column within pandas dataframe from int to string**

In [ ]:
df['A']=df['A'].astype(int)

> **Create new column using some calculations on some current columns**

**More than 2 cagetory**

In [ ]:
df['newCol'] = df['oldCol'].apply(lambda x: x + 1)

In [ ]:
def getSeason(month):
    """
    match season from month
    """
    
    if (month in [12,1,2]):
        season = 'winter' 
    elif (month in [3,4,5]):
        season = 'spring' 
    elif (month in [6,7,8]):
        season = 'summer'
    else:
        season = 'fall'

    return season   

In [ ]:
tran_df['Transaction Month'].apply(lambda x: getSeason(x))

**Two category using list comprehension**

In [ ]:
df_freq['frequency'] = ['F=1' if t == 1 else 'F>1' for t in df_freq['TranCount']]

> **create confusion table**

In [ ]:
y_actu = pd.Series([2, 0, 2, 2, 0, 1, 1, 2, 2, 0, 1, 2], name='Actual’) 
y_pred = pd.Series([0, 0, 2, 1, 0, 2, 1, 0, 2, 0, 2, 2], name='Predicted’) 
df_confusion = pd.crosstab(y_actu, y_pred)

> **Replace column values**

In [ ]:
df['female'] = df['female'].map({'female': 1, 'male': 0})

In [ ]:
df.loc[df.female != 'female', 'female'] = 0
df.loc[df.female == 'female', 'female'] = 1

> **convert strings to date**

In [3]:
pd.to_datetime('2018/02/26 00:00:00')

Timestamp('2018-02-26 00:00:00')

In [ ]:
df['Transaction Date'] = pd.to_datetime(df['Transaction Date'])

without date

In [5]:
pd.to_datetime('2018/02/26 00:00:00').date()

datetime.date(2018, 2, 26)

In [ ]:
df['Transaction Date'] = pd.to_datetime(df['Transaction Date']).apply(lambda x: x.date()) 

> **convert strings to time**

In [6]:
pd.to_datetime('06:44:50', format='%H:%M:%S')

Timestamp('1900-01-01 06:44:50')

In [7]:
pd.to_datetime('06:44:50', format='%H:%M:%S').time()

datetime.time(6, 44, 50)

In [ ]:
pd.to_datetime(df['Transaction Time'], format='%H:%M:%S').apply(lambda x: x.time()) 

> **get hour/date from datetime column, assigning to a new column**

In [ ]:
df['timestamp'] = df['timestamp'].astype('datetime64[ns]')
df['date'] = df['timestamp'].apply(lambda x: x.date()) 
df['weekday'] = df['timestamp'].apply(lambda x: x.weekday()) 
df['month'] = df['timestamp'].apply(lambda x: x.month)
df['hour'] = df['timestamp'].apply(lambda x: x.hour)
df['minute'] = df['timestamp'].apply(lambda x: x.minute)

[doc datetime](https://docs.python.org/3.3/library/datetime.html)

> **How to get the value from a timedelta?**

In [ ]:
age = datetime.timedelta(12045)

In [ ]:
print(age.days)
print(age.total_seconds())

[stackoverflow](https://stackoverflow.com/questions/27322362/how-to-get-the-value-from-a-timedelta)

> **rbind, cbind**

In [ ]:
# rbind
pd.concat([df1, df2])

# cbind
pd.concat([df1, df2], axis=1)

> **Pivot method**

In [4]:
df = pd.DataFrame({'foo': ['one','one','one','two','two','two'],
                       'bar': ['A', 'B', 'C', 'A', 'B', 'C'],
                       'baz': [1, 2, 3, 4, 5, 6]})
df

,bar,baz,foo
0,A,1,one
1,B,2,one
2,C,3,one
3,A,4,two
4,B,5,two
5,C,6,two


In [5]:
df.pivot(index='foo', columns='bar', values='baz')

bar,A,B,C
foo,,,
one,1,2,3
two,4,5,6


> **Gather columns into rows by using melt**

[doc](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.melt.html)

In [12]:
df = pd.DataFrame({'country': ['Taiwan','Japan'],
                    '2011': [1, 2],
                    '2012': [3, 4],
                    '2013': [5, 6],
                    '2014': [7, 8],                  
                  })
df

,2011,2012,2013,2014,country
0,1,3,5,7,Taiwan
1,2,4,6,8,Japan


In [15]:
df.melt(id_vars='country')#,var_name='year', value_name='value'

,country,variable,value
0,Taiwan,2011,1
1,Japan,2011,2
2,Taiwan,2012,3
3,Japan,2012,4
4,Taiwan,2013,5
5,Japan,2013,6
6,Taiwan,2014,7
7,Japan,2014,8


> **Bin values based on ranges with pandas**

In [ ]:
pd.cut(pd.Series([1,5,75,125]), [0, 50, 100,200])

[stackoverflow](https://stackoverflow.com/questions/31736671/bin-values-based-on-ranges-with-pandas)

> **Print full dataframe in jupyter notebook**

In [1]:
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

> **Getting the Row which has the max value in groups using groupby**

In [ ]:
idx = df.groupby(['Profile ID'])['count'].transform(max) == df['count']
df[idx]

[stackoverflow](https://stackoverflow.com/questions/15705630/python-getting-the-row-which-has-the-max-value-in-groups-using-groupby)

> **normalize columns in pandas**

In [ ]:
# Split data into response and predictors
y = train_binary_dummy['isNDF']
x = train_binary_dummy.drop('isNDF', axis=1)

In [ ]:
from sklearn.preprocessing import scale
def my_scaler(x):
    """standardize the predictors"""
    
    new_x = pd.DataFrame(scale(x, axis=0, with_mean=True, with_std=True, copy=True))
    new_x.columns = x.columns
    
    return new_x

In [ ]:
x = my_scaler(x)

> **Drop duplicate rows in Python Pandas**

In [ ]:
import pandas as pd
df = pd.DataFrame({"A":["foo", "foo", "foo", "bar"], "B":[0,1,1,1], "C":["A","A","B","A"]})
df.drop_duplicates(subset=['A', 'C'], keep=False)

[drop_duplicates doc](http://pandas.pydata.org/pandas-docs/version/0.17/generated/pandas.DataFrame.drop_duplicates.html)

[stackoverflow](https://stackoverflow.com/questions/23667369/drop-all-duplicate-rows-in-python-pandas)

> **Read and flatten json into pandas dataframe**

In [2]:
import pandas as pd
import json
from pandas.io.json import json_normalize

with open('../data/mpd.slice.0-999.json') as f:
    data = json.load(f)

song_df = json_normalize(data=data['playlists'], 
                         record_path='tracks', 
                         meta=['collaborative', 'duration_ms', 'modified_at', 'name', 
                               'num_albums', 'num_artists', 'num_edits', 'num_followers', 'num_tracks','pid', 'description'],
                         record_prefix='track_',
                         errors='ignore')

# record_path: the nested key that we want to flatten
# meta: the keys in the first level
# record_prefix: the prefix for the keys in the nested level
# error = ‘ignore’ : will ignore KeyError if keys listed in meta are not always present

* [JSON to pandas DataFrame](https://stackoverflow.com/questions/21104592/json-to-pandas-dataframe)
* [Document: pandas.io.json.json_normalize](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.io.json.json_normalize.html)
* [Quick Tutorial: Flatten Nested JSON in Pandas
](https://www.kaggle.com/jboysen/quick-tutorial-flatten-nested-json-in-pandas)

> **Get difference between two lists**

In [ ]:
temp1 = ['One', 'Two', 'Three', 'Four']
temp2 = ['One', 'Two']
list(set(temp1) - set(temp2))

> **group by count percentage for a categorical column**

In [7]:
df = pd.DataFrame({'foo': ['one','one','one','two','two','two'],
                       'bar': ['A', 'A', 'B', 'B', 'C', 'C'],
                       'value': [1, 2, 3, 4, 5, 6]})
df

,bar,foo,value
0,A,one,1
1,A,one,2
2,B,one,3
3,B,two,4
4,C,two,5
5,C,two,6


If we want to calculate the percentage of A using the sum of value for each group in foo, we can use `crosstab` from pandas

In [12]:
percentage_df = pd.crosstab(index=df['foo'],
                              columns=df['bar'],
                              values=df['value'],
                              aggfunc=np.sum,
                              dropna=False,
                              normalize='index') 
percentage_df = percentage_df.reset_index()
percentage_df.columns.name = None
percentage_df

,foo,A,B,C
0,one,0.5,0.500000,0.000000
1,two,0.0,0.266667,0.733333


[crosstab doc](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.crosstab.html)

> **pandas create new column based on values from other columns**

In [ ]:
def label_race (row):
    if row['eri_hispanic'] == 1:
        result = 'Hispanic'
    elif row['eri_afr_amer'] + row['eri_asian'] + row['eri_hawaiian'] + row['eri_nat_amer'] + row['eri_white'] > 1 :
        result = 'Two Or More'
    else:     
        result = 'White'
    return result

In [ ]:
df.apply (lambda row: label_race (row),axis=1)

[stackoverflow](https://stackoverflow.com/questions/26886653/pandas-create-new-column-based-on-values-from-other-columns)